In [5]:
# Import our dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import datetime
import re
import csv
import json
import os
# Read in "The IMbd Movies dataset"
# if this is read in as csv, each row is a string and dictionaries are not recognized.
IMDb_movies_df = pd.read_csv("IMDb_movies.csv", low_memory=False)
IMDb_movies_df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [6]:
print(IMDb_movies_df.describe())
print(IMDb_movies_df.columns)
#print(movies_df["release_date"].dtypes)

           duration      avg_vote         votes     metascore  \
count  85855.000000  85855.000000  8.585500e+04  13305.000000   
mean     100.351418      5.898656  9.493490e+03     55.896881   
std       22.553848      1.234987  5.357436e+04     17.784874   
min       41.000000      1.000000  9.900000e+01      1.000000   
25%       88.000000      5.200000  2.050000e+02     43.000000   
50%       96.000000      6.100000  4.840000e+02     57.000000   
75%      108.000000      6.800000  1.766500e+03     69.000000   
max      808.000000      9.900000  2.278845e+06    100.000000   

       reviews_from_users  reviews_from_critics  
count        78258.000000          74058.000000  
mean            46.040826             27.479989  
std            178.511411             58.339158  
min              1.000000              1.000000  
25%              4.000000              3.000000  
50%              9.000000              8.000000  
75%             27.000000             23.000000  
max          1

In [7]:
print(IMDb_movies_df["date_published"].dtypes)

object


In [8]:
print(IMDb_movies_df["year"].dtypes)

object


In [9]:
#IMDb_movies_df.rename(columns={'imdb_title_id':'imdb_id'}, inplace=True)
#IMDb_movies_df.head()

In [10]:
#IMDb_movies_df = IMDb_movies_df.drop(IMDb_movies_df[(IMDb_movies_df.country != 'USA') | (IMDb_movies_df.country != 'US')].index)
IMDb_movies_df = IMDb_movies_df.drop(IMDb_movies_df[IMDb_movies_df.country != 'USA'].index)
print(IMDb_movies_df.describe())

           duration      avg_vote         votes    metascore  \
count  28511.000000  28511.000000  2.851100e+04  6609.000000   
mean      93.050437      5.556084  1.450219e+04    52.966258   
std       18.576873      1.284809  6.853054e+04    17.942703   
min       42.000000      1.100000  9.900000e+01     1.000000   
25%       84.000000      4.800000  2.300000e+02    40.000000   
50%       91.000000      5.800000  6.060000e+02    53.000000   
75%      100.000000      6.500000  2.895000e+03    66.000000   
max      398.000000      9.700000  2.278845e+06   100.000000   

       reviews_from_users  reviews_from_critics  
count        28206.000000          26635.000000  
mean            62.591931             32.217383  
std            201.990401             63.215751  
min              1.000000              1.000000  
25%              8.000000              4.000000  
50%             17.000000             10.000000  
75%             43.000000             30.000000  
max           8869.0000

In [11]:
# clean up and change date_published to datetime
# save at each change to make reverting easy
date_p=IMDb_movies_df["date_published"]
date_p.str.strip()  #remove spaces
form1=r'(^[0-9]{4})'
rel_year = date_p.str.extract(f'({form1})')  #extract just year
rel_year["release_year"]=pd.to_numeric(rel_year[0])
print(rel_year.dtypes)
rel_year.drop([0,1],axis=1,inplace=True)
print(f'null rows: {rel_year.isnull().sum()}')
rel_year.dropna(how="any", inplace=True)
print(f'number of movies: {len(rel_year)}')
print(rel_year.dtypes)

0                object
1                object
release_year    float64
dtype: object
null rows: release_year    1
dtype: int64
number of movies: 28510
release_year    float64
dtype: object


In [12]:
rel_year=rel_year[rel_year["release_year"] > 1969]
year_counts=rel_year["release_year"].value_counts()
year_counts

2017.0    938
2018.0    909
2016.0    883
2015.0    838
2014.0    823
2013.0    797
2019.0    794
2012.0    695
2011.0    672
2008.0    619
2009.0    614
2010.0    607
2007.0    556
2006.0    543
2020.0    446
2005.0    432
1999.0    423
2000.0    392
2004.0    374
2001.0    363
2003.0    362
1997.0    360
1998.0    352
1988.0    344
2002.0    336
1989.0    331
1996.0    331
1993.0    314
1995.0    301
1987.0    292
1994.0    280
1991.0    275
1990.0    271
1992.0    254
1986.0    214
1984.0    196
1982.0    177
1975.0    176
1973.0    176
1985.0    174
1972.0    172
1979.0    169
1980.0    165
1970.0    164
1983.0    163
1971.0    162
1974.0    160
1981.0    156
1978.0    154
1977.0    153
1976.0    146
2021.0      3
Name: release_year, dtype: int64

In [15]:
len(rel_year)

20001

In [16]:
rel_year.head()

,release_year
569,2004.0
624,1970.0
808,1972.0
1034,2020.0
3140,2013.0


In [9]:
#rel_year.to_csv('IMDb_gt69.csv', index=False)

In [66]:
awards_df = pd.read_csv("awards.csv", low_memory=False)
awards_df.rename(columns={'const':'imdb_id'}, inplace=True)
awards_df.head()

,eventId,eventName,awardName,year,occurrence,winAnnouncementTime,categoryName,nomeneeNote,name,originalName,...,episodeNames,characterNames,isWinner,isPrimary,isSecondary,isPerson,isTitle,isCompany,imdb_id,notes
0,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Film,NaN,La belle noiseuse,NaN,...,NaN,NaN,True,True,False,False,True,False,tt0101428,NaN
1,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Film,NaN,Jacques Rivette,NaN,...,NaN,NaN,True,False,True,True,False,False,nm0729626,NaN
2,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Foreign Film,NaN,The Double Life of Véronique,La double vie de Véronique,...,NaN,NaN,True,True,False,False,True,False,tt0101765,Poland.
3,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Foreign Film,NaN,Krzysztof Kieslowski,NaN,...,NaN,NaN,True,False,True,True,False,False,nm0001425,Poland.
4,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Short,NaN,Le petit chat est mort,NaN,...,NaN,NaN,True,True,False,False,True,False,tt0195135,NaN


In [68]:
IMDb_movies_df[IMDb_movies_df['imdb_id'] == 'tt0101428']

,imdb_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics


In [18]:
IMDb_gt1969_df = pd.concat([IMDb_movies_df, rel_year], axis=1, ignore_index=False, join="inner")
len(IMDb_gt1969_df)

20001

In [19]:
IMDb_gt1969_df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,release_year
569,tt0018054,Il re dei re,The King of Kings,1927,2004,"Biography, Drama, History",160,USA,English,Cecil B. DeMille,...,Jesus Christ faces religious and political opp...,7.2,1890,$ 2500000,NaN,NaN,NaN,48.0,24.0,2004.0
624,tt0018742,Il cameraman,The Cameraman,1928,1970-12-19,"Comedy, Drama, Family",76,USA,English,"Edward Sedgwick, Buster Keaton",...,Hopelessly in love with a woman working at MGM...,8.1,10101,NaN,NaN,$ 1737460,NaN,61.0,49.0,1970.0
808,tt0020442,Io... e l'amore,Spite Marriage,1929,1972-01-05,Comedy,76,USA,English,"Edward Sedgwick, Buster Keaton",...,An unimpressive but well intending man is give...,7.0,1811,NaN,NaN,NaN,NaN,30.0,18.0,1972.0
1034,tt0021673,La bionda e l'avventuriero,Blonde Crazy,1931,2020-07-01,"Comedy, Crime, Drama",79,USA,English,Roy Del Ruth,...,The adventures of an egoistic con man and his ...,7.1,1561,NaN,NaN,NaN,NaN,34.0,16.0,2020.0
3140,tt0030881,Too Much Johnson,Too Much Johnson,1938,2013-10-09,Comedy,67,USA,English,Orson Welles,...,A woman has two lovers. When one man finds out...,5.9,534,NaN,NaN,NaN,NaN,10.0,16.0,2013.0


In [20]:
IMDb_gt1969_df.to_csv('IMDb_gt69.csv', index=False)

In [70]:
df=pd.read_csv("movies_metadata.csv", low_memory=False)
print(df.describe())
print(df.columns)
print(df["release_date"].dtypes)

            revenue       runtime  vote_average    vote_count
count  4.546000e+04  45203.000000  45460.000000  45460.000000
mean   1.120935e+07     94.128199      5.618207    109.897338
std    6.433225e+07     38.407810      1.924216    491.310374
min    0.000000e+00      0.000000      0.000000      0.000000
25%    0.000000e+00     85.000000      5.000000      3.000000
50%    0.000000e+00     95.000000      6.000000     10.000000
75%    0.000000e+00    107.000000      6.800000     34.000000
max    2.787965e+09   1256.000000     10.000000  14075.000000
Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
object


In [71]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [72]:
# clean up and change release_date to datetime
# save at each change to make reverting easy
r_date=df["release_date"]
r_date.str.strip()  #remove spaces
form1=r'(^[0-9]{4})'
rel_year2 = r_date.str.extract(f'({form1})')  #extract just year
rel_year2["release_year"]=pd.to_numeric(rel_year2[0])
print(rel_year2.dtypes)
rel_year2.drop([0,1],axis=1,inplace=True)
print(f'null rows: {rel_year2.isnull().sum()}')
rel_year2.dropna(how="any", inplace=True)
print(f'number of movies: {len(rel_year2)}')
print(rel_year2.dtypes)

0                object
1                object
release_year    float64
dtype: object
null rows: release_year    90
dtype: int64
number of movies: 45376
release_year    float64
dtype: object


In [73]:
rel_year2=rel_year2[rel_year2["release_year"] > 1989]
year_counts2=rel_year2["release_year"].value_counts()
year_counts2

2014.0    1974
2015.0    1905
2013.0    1889
2012.0    1722
2011.0    1667
2016.0    1604
2009.0    1586
2010.0    1501
2008.0    1473
2007.0    1320
2006.0    1270
2005.0    1125
2004.0     992
2002.0     905
2003.0     882
2001.0     865
2000.0     789
1999.0     723
1998.0     722
1997.0     661
1996.0     633
1995.0     599
1994.0     544
2017.0     532
1993.0     489
1992.0     453
1990.0     427
1991.0     426
2018.0       5
2020.0       1
Name: release_year, dtype: int64

In [74]:
# production_companies is a string that looks like a list of dictionaries
# filter rows to keep only those with 'iso_3166_1' = 'US'
pd_countries=df["production_countries"]  # pd_countries is a series
form1=r"'iso_3166_1':\s*('US')"
country = pd_countries.str.extract(f'({form1})')
country.drop([0],axis=1,inplace=True)
print(country.notnull().sum())
country.dropna(how="any", inplace=True)
len(country)

1    21153
dtype: int64


21153

In [75]:
movies_year_df = pd.concat([df, rel_year2, country], axis=1, ignore_index=False, join="inner")
len(movies_year_df)

12906

In [77]:
merge_movies_df = pd.merge(IMDb_gt1990_df, movies_year_df, on='imdb_id', suffixes=['_TM','_MD'])
print(merge_movies_df.describe())

          duration     avg_vote         votes    metascore  \
count  6268.000000  6268.000000  6.268000e+03  3750.000000   
mean     99.207562     5.757291  4.828954e+04    50.886933   
std      15.206566     1.136640  1.251395e+05    17.036230   
min      45.000000     1.300000  1.020000e+02     1.000000   
25%      90.000000     5.100000  1.580750e+03    38.250000   
50%      96.000000     5.900000  6.732000e+03    51.000000   
75%     106.000000     6.500000  3.743975e+04    64.000000   
max     271.000000     9.300000  2.278845e+06   100.000000   

       reviews_from_users  reviews_from_critics  release_year_TM  \
count         6258.000000           6209.000000      6268.000000   
mean           150.815117             78.927686      2005.970006   
std            311.219300            101.831307         8.019208   
min              1.000000              1.000000      1990.000000   
25%             21.000000             15.000000      1999.000000   
50%             53.000000        

In [78]:
merge_movies_df.columns

Index([              'imdb_id',              'title_TM',
           'original_title_TM',                  'year',
              'date_published',                 'genre',
                    'duration',               'country',
                    'language',              'director',
                      'writer',    'production_company',
                      'actors',           'description',
                    'avg_vote',                 'votes',
                   'budget_TM',      'usa_gross_income',
       'worlwide_gross_income',             'metascore',
          'reviews_from_users',  'reviews_from_critics',
             'release_year_TM',                 'adult',
       'belongs_to_collection',             'budget_MD',
                      'genres',              'homepage',
                          'id',     'original_language',
           'original_title_MD',              'overview',
                  'popularity',           'poster_path',
        'production_companies',

In [32]:
IMDb_movies69_df = pd.read_csv("IMDb_gt69.csv", low_memory=False)
IMDb_movies69_df.head(100)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,release_year
0,tt0018054,Il re dei re,The King of Kings,1927,2004,"Biography, Drama, History",160,USA,English,Cecil B. DeMille,...,Jesus Christ faces religious and political opp...,7.2,1890,$ 2500000,NaN,NaN,NaN,48.0,24.0,2004.0
1,tt0018742,Il cameraman,The Cameraman,1928,1970-12-19,"Comedy, Drama, Family",76,USA,English,"Edward Sedgwick, Buster Keaton",...,Hopelessly in love with a woman working at MGM...,8.1,10101,NaN,NaN,$ 1737460,NaN,61.0,49.0,1970.0
2,tt0020442,Io... e l'amore,Spite Marriage,1929,1972-01-05,Comedy,76,USA,English,"Edward Sedgwick, Buster Keaton",...,An unimpressive but well intending man is give...,7.0,1811,NaN,NaN,NaN,NaN,30.0,18.0,1972.0
3,tt0021673,La bionda e l'avventuriero,Blonde Crazy,1931,2020-07-01,"Comedy, Crime, Drama",79,USA,English,Roy Del Ruth,...,The adventures of an egoistic con man and his ...,7.1,1561,NaN,NaN,NaN,NaN,34.0,16.0,2020.0
4,tt0030881,Too Much Johnson,Too Much Johnson,1938,2013-10-09,Comedy,67,USA,English,Orson Welles,...,A woman has two lovers. When one man finds out...,5.9,534,NaN,NaN,NaN,NaN,10.0,16.0,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,tt0065405,Angels Die Hard,Angels Die Hard,1970,1970-07-08,"Action, Drama",86,USA,English,Richard Compton,...,Gang of bikers try to save people in a mining ...,5.4,136,NaN,NaN,NaN,NaN,4.0,9.0,1970.0
96,tt0065407,Le 5 facce della violenza,The Animals,1970,1971-10-01,Western,86,USA,English,Ron Joy,...,A woman tracks down the three men who raped he...,3.6,281,NaN,NaN,NaN,NaN,10.0,1.0,1971.0
97,tt0065421,Gli Aristogatti,The AristoCats,1970,1971-11-13,"Animation, Adventure, Comedy",78,USA,English,Wolfgang Reitherman,...,"With the help of a smooth talking tomcat, a fa...",7.1,91085,$ 4000000,$ 35452658,$ 35459543,66.0,141.0,113.0,1971.0
98,tt0065424,Sensualità morbosa,Sweet Kill,1972,1973-01-01,"Horror, Thriller",85,USA,English,Curtis Hanson,...,Horror and suspense in the story of a psychoti...,5.0,245,$ 110000,NaN,NaN,NaN,14.0,10.0,1973.0


In [22]:
IMDb_movies69_df.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics', 'release_year'],
      dtype='object')

In [23]:
IMDb_movies69_df["actors"] #thinking there are too many actors to bin or assign a value, so may need to drop this field

0        H.B. Warner, Dorothy Cumming, Ernest Torrence,...
1        Buster Keaton, Marceline Day, Harold Goodwin, ...
2        Buster Keaton, Dorothy Sebastian, Edward Earle...
3        James Cagney, Joan Blondell, Louis Calhern, No...
4        Joseph Cotten, Virginia Nicolson, Edgar Barrie...
                               ...                        
19996    Dominique Provost-Chalkley, Jessica Clark, Emi...
19997    Faizon Love, Kirk Fox, Tony Todd, Cassie Clare...
19998    Stephen Lang, William Sadler, Fred Williamson,...
19999    David Thorpe, John Rhys-Davies, Kristyn Getty,...
20000    Ed Helms, Taraji P. Henson, Terrence Little Ga...
Name: actors, Length: 20001, dtype: object

In [26]:
IMDb_movies69_df["genre"].tail(50) #break into 3 columns or just grab first one listed

19951                          Sci-Fi
19952                          Horror
19953         Action, Fantasy, Horror
19954                 Comedy, Musical
19955       Adventure, Drama, Fantasy
19956         Action, Crime, Thriller
19957                          Horror
19958                       Animation
19959                          Horror
19960                          Comedy
19961                        Thriller
19962                  Action, Sci-Fi
19963    Animation, Action, Adventure
19964                           Drama
19965          Comedy, Drama, Romance
19966                           Drama
19967                          Comedy
19968                           Drama
19969                 Comedy, Romance
19970                Action, Thriller
19971                           Drama
19972          Comedy, Drama, Romance
19973    Animation, Adventure, Comedy
19974          Comedy, Drama, Romance
19975                Horror, Thriller
19976                Horror, Thriller
19977       

In [28]:
IMDb_movies69_df["director"].head(50)

0                     Cecil B. DeMille
1       Edward Sedgwick, Buster Keaton
2       Edward Sedgwick, Buster Keaton
3                         Roy Del Ruth
4                         Orson Welles
5                        James Mangold
6                    Vincente Minnelli
7                        Samuel Fuller
8                           Del Tenney
9                       Shirley Clarke
10                    Jacques Tourneur
11    Harold Daniels, Reginald Le Borg
12                       Monte Hellman
13           Carl Monson, Dwayne Avery
14              Harry Hope, Lee Sholem
15                        Gus Trikonis
16                       Monte Hellman
17                       Arthur Hiller
18                       Bruce Kessler
19                       Frank Telford
20              Dan Martin, Byron Mabe
21                         Andy Warhol
22                         Jim McBride
23                Francis Ford Coppola
24                      Paul Morrissey
25                       

In [29]:
IMDb_movies69_df.nunique()

imdb_title_id            20001
title                    19522
original_title           19247
year                        67
date_published            8058
genre                      680
duration                   153
country                      1
language                   532
director                 11050
writer                   16487
production_company       10302
actors                   19964
description              19931
avg_vote                    85
votes                     7762
budget                     654
usa_gross_income          7042
worlwide_gross_income     7375
metascore                   98
reviews_from_users         898
reviews_from_critics       491
release_year                52
dtype: int64

In [30]:
director = IMDb_movies69_df.director.value_counts()

In [31]:
director #lets explore binning all directors with < 32

Fred Olen Ray          50
David DeCoteau         39
Woody Allen            36
Clint Eastwood         34
Jim Wynorski           32
                       ..
Antonia Bogdanovich     1
Rory Douglas Abel       1
Larry Thomas            1
Tim Metcalfe            1
Carlos Ferrer           1
Name: director, Length: 11050, dtype: int64

In [35]:
genre = IMDb_movies69_df.genre.value_counts()
genre.tail(50)

Biography, Crime, History       1
Sci-Fi, Western                 1
History, War                    1
Horror, Reality-TV, Thriller    1
Drama, Fantasy, Music           1
Adventure, Drama, Sport         1
Horror, Musical, Sci-Fi         1
Comedy, Musical, Western        1
Family, Musical, Romance        1
Action, Crime, Western          1
Horror, Action                  1
Western, Horror                 1
Action, Adventure, Biography    1
Mystery, Comedy                 1
Action, Thriller, Mystery       1
Sci-Fi, Drama, Fantasy          1
Horror, Drama, Thriller         1
Family, Drama, Comedy           1
Action, Family, Sci-Fi          1
Crime, Comedy, Fantasy          1
Fantasy, Horror, Musical        1
Action, Western, Fantasy        1
Western, Drama                  1
Drama, Action, Crime            1
Adventure, Crime, Action        1
Family, Drama, Western          1
Action, Music                   1
Comedy, Family, Mystery         1
Family, War, Action             1
Family, Comedy

In [37]:
writer = IMDb_movies69_df.writer.value_counts()
writer.tail(50)

Ray Bradbury, Roger Lay Jr.            1
Johnno Zee                             1
Mitchell Altieri, Phil Flores          1
Prince                                 1
Jerry Sterner, Alvin Sargent           1
Kelly Marcel, E.L. James               1
Brian A. Miller                        1
Jonathan Butler                        1
Lynn Drzick                            1
Graham Greene, Guy Elmes               1
Dianne Houston                         1
Kazeem Molake, Kazeem Molake           1
Kelly Goodner, Brenda Jackson          1
Timothy Barton                         1
Robert J. Gibbs, Whit Hertford         1
Howard Brookner, Colman deKay          1
Matthew Porterfield                    1
Richard Friedman, Rick Marx            1
Milan Stitt, Milan Stitt               1
Kenneth Dorward                        1
Ana Asensio                            1
E.G. Valens, David Seltzer             1
Michael Zettler, Shelly Altman         1
Jason Momoa, Robert Homer Mollohan     1
Robert Foreman, 